In [1]:
import zmq
import cv2
import numpy as np
from open3d import *
import PyUrComu.UrCommunication as UrCommunication
from scipy.spatial.transform import Rotation as R
import time
from colorama import Fore, Back, Style
import pickle as pkl
import open3d as o3d
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import euclidean_distances
import traceback
from tqdm import tqdm
import random

In [2]:
### global variables and elements
cv2.namedWindow('image_3d_to_display', cv2.WINDOW_AUTOSIZE)
calibration_TCP=[0,0,0.23]
calibration_waypoints=[]
calibration_waypoints_final=[]

calibration_position_0=[-0.415, -0.275, 0.025]
calibration_position_1=[-0.415, -0.300, 0.025]
calibration_position_2=[-0.415, -0.300, 0.015]
calibration_position_3=[-0.415, -0.275, 0.015]

calibration_position_4=[-0.475, -0.275, 0.025]
calibration_position_5=[-0.475, -0.300, 0.025]
calibration_position_6=[-0.475, -0.300, 0.015]
calibration_position_7=[-0.475, -0.275, 0.015]

picked_points=[]
picked_points_3d_camera=[]
picked_points_3d_robot=[]
picked_points_3d_robot_current=[]
reference_point_in_camera_coordinates=None
reference_point=None
lower_point=None
frame_num=0
points_for_set_tcp=[]

In [3]:
### Set start position and set TCP
start_position_tcp=[-0.44365, -0.28609, 0.01423]
start_position_rotation_rotation_vector=[0.003, 3.134,-0.001]

### rotations used for TCP_calibration
calibration_rotation_rpy_1=[3.141,0.370,3.139]
calibration_rotation_rpy_2=[-3.140,-0.337,3.139]
calibration_rotation_rpy_3=[-2.958,0.166,3.141]

In [4]:
### Functions
def take_picture(socket):
    socket.send(b"tcp_calibration")
    message = socket.recv()
    socket.send (b"1")
    message = socket.recv()
    print ("Received reply from server:", message)

In [5]:
def find_lower_pixel(image_3d):
    last_non_zero_height=0
    x_position_of_last_pixel=0
    non_zero_indexes=[]
    lower_y_limit=700 
    for y in range(lower_y_limit):
        line=image_3d[y,:]
        num=line.any(axis=-1).sum()  
        if num!=0:
            last_non_zero_height=y

            non_zero_indexes=np.nonzero(line.any(axis=-1))[0]

    print("last_non_zero_height: ",last_non_zero_height)
    print("non_zero_indexes: ",non_zero_indexes)
    print("non_zero_indexes[int(len(non_zero_indexes)/2)]: ",non_zero_indexes[int(len(non_zero_indexes)/2)])
    point_3d=np.round(image_3d[last_non_zero_height,non_zero_indexes[int(len(non_zero_indexes)/2)]],7)
    point_3d[2]=point_3d[2]-0.0002 # the value 0.0002 value is chosen according the shape of needle, 
    # used for calibration. Needle itselfe has diametr 0.65 mm. 0.2mm is where 
    # the camera estimates tip of the needle according our observations
    point_2d=(non_zero_indexes[int(len(non_zero_indexes)/2)],last_non_zero_height)
    return point_3d, point_2d


In [6]:
def calculate_waypoints(point_1,point_2,step):
    waypoints=[]
    step_x=(point_2[0]-point_1[0])/step
    step_y=(point_2[1]-point_1[1])/step
    step_z=(point_2[2]-point_1[2])/step
    for i in range(0,step):
        temp_point=[point_1[0]+step_x*i,point_1[1]+step_y*i,point_1[2]+step_z*i]
        waypoints.append(temp_point)
    waypoints.append(point_2)
    return waypoints

In [7]:
def get_images(socket):
    take_picture(socket)
    wire_1_mat=cv2.imread("images/image_3d.exr", cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    grayimage=cv2.imread("images/graymat.jpg", 0)
    grayimage = cv2.equalizeHist(grayimage)
    grayimage=cv2.cvtColor(grayimage, cv2.COLOR_GRAY2BGR)
    print("wire_1.exr loaded. Shape: ", wire_1_mat.shape)
    return wire_1_mat,grayimage

In [8]:
# taken from https://github.com/nghiaho12/rigid_transform_3D/blob/master/rigid_transform_3D.py
def rigid_transform_3D(A, B):
    assert len(A) == len(B)

    num_rows, num_cols = A.shape;

    if num_rows != 3:
        raise Exception("matrix A is not 3xN, it is {}x{}".format(num_rows, num_cols))

    [num_rows, num_cols] = B.shape;
    if num_rows != 3:
        raise Exception("matrix B is not 3xN, it is {}x{}".format(num_rows, num_cols))

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # subtract mean
    Am = A - np.tile(centroid_A, (1, num_cols))
    Bm = B - np.tile(centroid_B, (1, num_cols))

    H = Am * np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T * U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...\n");
        Vt[2,:] *= -1
        R = Vt.T * U.T

    t = -R*centroid_A + centroid_B

    return R, t


In [9]:
def create_transformation_matrix(picked_points_3d_camera,picked_points_3d_robot): #TODO rewrite this function with NP Arrays
    A=np.asmatrix(picked_points_3d_camera).T
    B=np.asmatrix(np.asanyarray(picked_points_3d_robot)).T
    r,t=rigid_transform_3D(A,B)
    temp_mat_1=np.concatenate((r,t),axis=1)
    temp_mat_2=np.mat((0,0,0,1))
    transformation_matrix=np.array(np.concatenate((temp_mat_1,temp_mat_2),axis=0)) #TODO fix that function to avoid matrixes
    print("Transformation matrix created.")
    print(transformation_matrix)
    transformation_matrix_analyze=R.from_matrix(transformation_matrix [:3,:3])
    transformation_matrix_analyze=transformation_matrix_analyze.as_euler("xyz")
    print("rotations rX,rY,rZ: ",np.degrees(transformation_matrix_analyze[0]), \
          np.degrees(transformation_matrix_analyze[1]),np.degrees(transformation_matrix_analyze[2]))
    return transformation_matrix

In [10]:
def paint_image():
    # grab references to the global variables
    global last_point
    global reference_point_3d
    global last_point_3d
    global wire_1_mat
    global reference_point
    global paint_reference
    global grayimage
    global transformation_matrix
    global lower_point
    global reference_point_in_camera_coordinates
    image_to_show=grayimage.copy()
    image_to_show = cv2.putText(image_to_show, 'TCP: '+str(round(ur.ActualTcpPos[0],4))+", "+str(round(ur.ActualTcpPos[1],4))+", "+str(ur.ActualTcpPos[2]), (20,40), cv2.FONT_HERSHEY_SIMPLEX , 0.7, (0, 255, 0) , 2, cv2.LINE_AA)
    image_to_show = cv2.putText(image_to_show, 'lower point: '+str(reference_point_in_camera_coordinates),(20,120), cv2.FONT_HERSHEY_SIMPLEX , 0.7, (0, 255, 255) , 2, cv2.LINE_AA)
    image_to_show = cv2.putText(image_to_show, 'frame_num: '+str(frame_num), (20,140), cv2.FONT_HERSHEY_SIMPLEX , 0.7, (0, 255, 0) , 2, cv2.LINE_AA)
    print("lower_point: ",lower_point)
    cv2.circle(image_to_show,lower_point,4,(0,0,255),-1)
    dim=(int(image_to_show.shape[1]/2),int(image_to_show.shape[0]/2))
    image_to_show=cv2.resize(image_to_show,dim )
    cv2.imshow("image_3d_to_display",image_to_show)
    cv2.waitKey(10)

In [11]:
def go_to_startposition(ur,start_position_3d,actual_rotation,transformation_matrix,socket):
    global grayimage
    global wire_1_mat
    global lower_point
    global reference_point_in_camera_coordinates
    actual_position_3d_rob_coord=None

    start_position_3d_t=np.append(start_position_3d,1).reshape((-1,1))
    print("\n start_position_3d_t",start_position_3d_t)
    start_position_3d_rob_coord_t=np.dot(transformation_matrix[:4,:4],start_position_3d_t)
    print("\n start_position_3d_rob_coord_t",start_position_3d_rob_coord_t)

    while(1):

        # Get 3d and color frames
        wire_1_mat,grayimage=get_images(socket)
        actual_position_3d,lower_point = find_lower_pixel(wire_1_mat)
        reference_point_in_camera_coordinates=actual_position_3d
        actual_position_3d_t=np.append(actual_position_3d,1).reshape((-1,1))
        actual_position_3d_rob_coord=np.dot(transformation_matrix[:4,:4],actual_position_3d_t)
        difference=np.linalg.norm(actual_position_3d_rob_coord-start_position_3d_rob_coord_t)
        print("difference: ",difference)
        if difference<0.00005: # stop criteria. 0.005mm difference from given position
            break
        diff_vector=(actual_position_3d_rob_coord-start_position_3d_rob_coord_t).reshape(1,-1)[0]
        # Get actual robot data
        ur.formatSharedData()
        print("TCP Position in Robot Coordinates: ",[ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]])
        print("next point: ",ur.ActualTcpPos[0]-diff_vector[0]/2,ur.ActualTcpPos[1]-diff_vector[1]/2,ur.ActualTcpPos[2]-diff_vector[2]/2)
        print("start_position_3d_rob_coord_t\n",start_position_3d_rob_coord_t,"\nactual_position_3d_rob_coord: \n",actual_position_3d_rob_coord)        
        actual_position_3d_rob_coord=actual_position_3d_rob_coord.reshape(1,-1)[0]
        print("diff_vector",diff_vector)
        # Move to next point and get coordinates
        ur.sendMoveL([ur.ActualTcpPos[0]-diff_vector[0]/2,ur.ActualTcpPos[1]-diff_vector[1]/2,ur.ActualTcpPos[2]-diff_vector[2]/2, \
                      actual_rotation[0], \
                      actual_rotation[1], \
                      actual_rotation[2]],
                     calibration_TCP, 30 / 1000, 0.2, 0.0)
        ur.waitForMessage()
        paint_image()
        cv2.waitKey(1000)
    return [ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2],ur.ActualTcpPos[3],ur.ActualTcpPos[4],ur.ActualTcpPos[5]]

In [12]:
### Create array with path points for calibration

# Path for precalibration
step=2
calibration_waypoints.extend(calculate_waypoints(calibration_position_0,calibration_position_7,step))
calibration_waypoints.extend(calculate_waypoints(calibration_position_7,calibration_position_6,step))
calibration_waypoints.extend(calculate_waypoints(calibration_position_6,calibration_position_1,step))
calibration_waypoints.extend(calculate_waypoints(calibration_position_1,calibration_position_0,step))
calibration_waypoints.extend(calculate_waypoints(calibration_position_0,calibration_position_7,step))
calibration_waypoints.extend(calculate_waypoints(calibration_position_7,calibration_position_1,step))

# Calibration path for calibration with estimated tcp
step=6
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_0,calibration_position_1,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_2,calibration_position_3,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_3,calibration_position_0,step))

calibration_waypoints_final.extend(calculate_waypoints(calibration_position_5,calibration_position_6,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_6,calibration_position_7,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_7,calibration_position_0,step))


calibration_waypoints_final.extend(calculate_waypoints(calibration_position_0,calibration_position_4,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_1,calibration_position_5,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_2,calibration_position_6,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_3,calibration_position_7,step))

calibration_waypoints_final.extend(calculate_waypoints(calibration_position_0,calibration_position_6 ,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_1,calibration_position_7 , step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_2,calibration_position_4 ,step))
calibration_waypoints_final.extend(calculate_waypoints(calibration_position_3,calibration_position_5 ,step))

random.shuffle(calibration_waypoints_final)
print(calibration_waypoints_final)

# initial points
calibration_waypoints_final=calculate_waypoints(calibration_position_0,calibration_position_1,1)+calibration_waypoints_final
calibration_waypoints_final=calculate_waypoints(calibration_position_6,calibration_position_4,1)+calibration_waypoints_final


[[-0.415, -0.275, 0.025], [-0.415, -0.2791666666666667, 0.015], [-0.415, -0.275, 0.025], [-0.46499999999999997, -0.275, 0.025], [-0.475, -0.2791666666666667, 0.015], [-0.44499999999999995, -0.2875, 0.02], [-0.475, -0.3, 0.015], [-0.45499999999999996, -0.275, 0.018333333333333333], [-0.425, -0.2791666666666667, 0.016666666666666666], [-0.46499999999999997, -0.2791666666666667, 0.016666666666666666], [-0.415, -0.2791666666666667, 0.025], [-0.425, -0.275, 0.015], [-0.44499999999999995, -0.2875, 0.02], [-0.415, -0.2875, 0.015], [-0.46499999999999997, -0.3, 0.025], [-0.475, -0.2875, 0.015], [-0.415, -0.275, 0.016666666666666666], [-0.435, -0.3, 0.015], [-0.435, -0.275, 0.015], [-0.415, -0.275, 0.021666666666666667], [-0.435, -0.2916666666666667, 0.021666666666666667], [-0.425, -0.275, 0.025], [-0.415, -0.2875, 0.025], [-0.44499999999999995, -0.3, 0.025], [-0.475, -0.3, 0.023333333333333334], [-0.415, -0.275, 0.025], [-0.415, -0.2833333333333333, 0.015], [-0.415, -0.3, 0.015], [-0.415, -0.27

In [13]:
### Start UR communication
ur = UrCommunication.UrRobot('Parameters.json')
ur.connectUr()
ur.startUpdateProcess("data")


In [14]:
start_position_rotation=R.from_rotvec(start_position_rotation_rotation_vector)
start_position_rotation_rpy=start_position_rotation.as_euler("xyz")
print(start_position_rotation_rpy)

[-3.14096175  0.00759167  3.13968056]


In [15]:
### Move to start position and initialize tcp variables
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              start_position_rotation_rpy[0], \
              start_position_rotation_rpy[1], \
              start_position_rotation_rpy[2]],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              start_position_rotation_rpy[0], \
              start_position_rotation_rpy[1], \
              start_position_rotation_rpy[2]-0.01],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              start_position_rotation_rpy[0], \
              start_position_rotation_rpy[1], \
              start_position_rotation_rpy[2]],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()

b'Ready'
b'Ready'
b'Ready'


b'Ready'

In [16]:
### connect to zmq and take pictures
context = zmq.Context()
socket = context.socket(zmq.REQ)
port = "5556"
socket.connect ("tcp://localhost:%s" % port)

# Precalibration

In [17]:
### Step 1: You are supposed to set TCP on robot somehow near to the needle end. Something around +-10mm accuracy is enought. ###

### Streaming loop for estimation of transformation_matrix for further TCP calibration.
try:
    for waypoint in calibration_waypoints:

        # Move to next point and get coordinates
        ur.sendMoveL([waypoint[0],waypoint[1],waypoint[2], \
                      start_position_rotation_rpy[0], \
                      start_position_rotation_rpy[1], \
                      start_position_rotation_rpy[2]],
                     calibration_TCP, 30 / 1000, 0.2,0)
        ur.waitForMessage()

        # Get actual robot data
        ur.formatSharedData()

        # Give robot time to stop vibration
        time.sleep(0.5)

        # Get 3d and color frames
        wire_1_mat,grayimage=get_images(socket)
        reference_point_in_camera_coordinates,lower_point= find_lower_pixel(wire_1_mat)
        print("reference_point_in_camera_coordinates: ",reference_point_in_camera_coordinates,"\n")

        # Append next detected point to arrays with both coordinate systems
        picked_points_3d_camera.append(reference_point_in_camera_coordinates)

        picked_points_3d_robot.append([ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]])
        print("picked_points_3d_robot: ",picked_points_3d_robot[-1],"\n\n")
        paint_image()
        frame_num+=1

    if picked_points_3d_camera==[] or picked_points_3d_robot==[]:
        print("Arrays with points are empty. Probably aruco symbol is out of FOV. Calibration is impossible")
        exit(1)

    temporal_transformation_matrix=create_transformation_matrix(picked_points_3d_camera, picked_points_3d_robot)
finally:
    print("step 1 is crashed")

b'Ready'
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  354
non_zero_indexes:  [1570 1571]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1571
reference_point_in_camera_coordinates:  [ 0.0330187 -0.0243073  0.222208 ] 

picked_points_3d_robot:  [-0.4151362462931921, -0.2751339918531293, 0.025173830885369064] 


lower_point:  (1571, 354)
b'Ready'
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  424
non_zero_indexes:  [1088]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1088
reference_point_in_camera_coordinates:  [ 0.0026648 -0.0197878  0.2210906] 

picked_points_3d_robot:  [-0.4450168240445052, -0.2750896328313275, 0.020062826289478003] 


lower_point:  (1088, 424)
b'Ready'
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  507
non_zero_indexes:  [603]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  603
reference_point_in_camera_coo

# TCP estimation

In [18]:
### Step 2: Robot moves needle end to the TCP calibration position and camera estimates it in its coordinate system. ###

# Move to calibration position
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              start_position_rotation_rpy[0], \
              start_position_rotation_rpy[1], \
              start_position_rotation_rpy[2]],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()

# Get actual robot data
ur.formatSharedData()

# Give robot time to stop vibration
time.sleep(0.5)
points_for_set_tcp.append([ur.ActualTcpPos])

b'Ready'


In [19]:
# Get 3d, gray frames and estimate first point for TCP calibration position for needle end
wire_1_mat,grayimage=get_images(socket)
calibration_position_for_needle_end,lower_point= find_lower_pixel(wire_1_mat)
print("calibration_position_for_needle_end: ",calibration_position_for_needle_end,"\n")

Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  511
non_zero_indexes:  [1117]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1117
calibration_position_for_needle_end:  [ 0.0042571 -0.0136484  0.2102571] 



In [20]:
paint_image()
line=None
while (line!="ready"):
    line=input(Fore.GREEN+"Switch to local mode on robot, create new TCP, and set first point.\n Then switch back to remote mode and input 'ready' to proceed:\n"+Style.RESET_ALL)


lower_point:  (1117, 511)
Switch to local mode on robot, create new TCP, and set first point.
 Then switch back to remote mode and input 'ready' to proceed:
ready


In [21]:
### calibration_rotation_rpy_1
# Move to calibration position
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              calibration_rotation_rpy_1[0], \
              calibration_rotation_rpy_1[1], \
              calibration_rotation_rpy_1[2]],
             calibration_TCP, 30 / 1000, 10,0.0)
ur.waitForMessage()
#start_position_rotation_rpy   calibration_rotation_rpy_1
points_for_set_tcp.append(go_to_startposition(ur,calibration_position_for_needle_end,calibration_rotation_rpy_1,temporal_transformation_matrix,socket))

# Get actual robot data
ur.formatSharedData()



b'Ready'

 start_position_3d_t [[ 0.0042571]
 [-0.0136484]
 [ 0.2102571]
 [ 1.       ]]

 start_position_3d_rob_coord_t [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]]
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  512
non_zero_indexes:  [1130]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1130
difference:  0.000774309640164269
TCP Position in Robot Coordinates:  [-0.44365990919151277, -0.28609202408049866, 0.014223620187711727]
next point:  -0.44404485624609946 -0.28606010710925023 0.014249810392448704
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.4429072 ]
 [-0.28620899]
 [ 0.01440972]
 [ 1.        ]]
diff_vector [ 7.69894109e-04 -6.38339425e-05 -5.23804095e-05  0.00000000e+00]
b'Ready'
lower_point:  (1130, 512)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  512
non_zero_indexes:  [1

In [22]:
paint_image()
line=None
while (line!="ready"):
    line=input(Fore.GREEN+"Switch to local mode on robot, and set second point.\n Then switch back to remote mode and input 'ready' to proceed:\n"+Style.RESET_ALL)


lower_point:  (1117, 511)
Switch to local mode on robot, and set second point.
 Then switch back to remote mode and input 'ready' to proceed:
ready


In [23]:
### calibration_rotation_rpy_2
# Move to calibration position
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              calibration_rotation_rpy_2[0], \
              calibration_rotation_rpy_2[1], \
              calibration_rotation_rpy_2[2]],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()
points_for_set_tcp.append(go_to_startposition(ur,calibration_position_for_needle_end,calibration_rotation_rpy_2,temporal_transformation_matrix,socket))
# Get actual robot data
ur.formatSharedData()

b'Ready'

 start_position_3d_t [[ 0.0042571]
 [-0.0136484]
 [ 0.2102571]
 [ 1.       ]]

 start_position_3d_rob_coord_t [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]]
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  507
non_zero_indexes:  [1102 1103]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1103
difference:  0.0008678991413994695
TCP Position in Robot Coordinates:  [-0.4436741498908963, -0.2860724965368974, 0.014220317915775799]
next point:  -0.4432557228499254 -0.28604907694065224 0.014107701026249777
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44451395]
 [-0.28619199]
 [ 0.01468733]
 [ 1.        ]]
diff_vector [-8.36854082e-04 -4.68391925e-05  2.25233779e-04  0.00000000e+00]
b'Ready'
lower_point:  (1103, 507)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  507
non_zero_indexes: 

b'Ready'
lower_point:  (1116, 512)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  510
non_zero_indexes:  [1116]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1116
difference:  0.00013596661058688347
TCP Position in Robot Coordinates:  [-0.44284209217317116, -0.2860300286740958, 0.014019113914826603]
next point:  -0.4428093436646159 -0.28597700150773103 0.01399195908812216
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44374259]
 [-0.28625121]
 [ 0.01451641]
 [ 1.        ]]
diff_vector [-6.54970171e-05 -1.06054333e-04  5.43096534e-05  0.00000000e+00]
b'Ready'
lower_point:  (1116, 510)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  510
non_zero_indexes:  [1117 1118]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1118
difference:  0.00010057324983069971
TCP Position in Robot Coordinates:  [-0.4427901

b'Ready'
lower_point:  (1115, 516)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  513
non_zero_indexes:  [1116]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1116
difference:  0.00014495051736182994
TCP Position in Robot Coordinates:  [-0.4428271079583065, -0.2859518891604602, 0.013823505811051906]
next point:  -0.4427968070313223 -0.2859263793578531 0.01388419985454637
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44373769]
 [-0.28619617]
 [ 0.01434071]
 [ 1.        ]]
diff_vector [-6.06018540e-05 -5.10196052e-05 -1.21388087e-04  0.00000000e+00]
b'Ready'
lower_point:  (1116, 513)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  512
non_zero_indexes:  [1118]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1118
difference:  8.488624225527639e-05
TCP Position in Robot Coordinates:  [-0.442799341676910

b'Ready'
lower_point:  (1120, 507)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  515
non_zero_indexes:  [1115]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1115
difference:  0.00026955278367849406
TCP Position in Robot Coordinates:  [-0.44286157423605743, -0.2860180981994489, 0.013818728612088577]
next point:  -0.4428026833943699 -0.2860010378490526 0.013938751480771562
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44379487]
 [-0.28617927]
 [ 0.01422205]
 [ 1.        ]]
diff_vector [-1.17781683e-04 -3.41207008e-05 -2.40045737e-04  0.00000000e+00]
b'Ready'
lower_point:  (1115, 515)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  511
non_zero_indexes:  [1117]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1117
difference:  7.97812518378312e-05
TCP Position in Robot Coordinates:  [-0.44280080570159

b'Ready'
lower_point:  (1118, 510)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  512
non_zero_indexes:  [1116]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1116
difference:  9.68755991626565e-05
TCP Position in Robot Coordinates:  [-0.44281169449186, -0.28596500106896255, 0.013930011019076927]
next point:  -0.44278123766954086 -0.2859434420057838 0.013960894820613618
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44373801]
 [-0.28618827]
 [ 0.01440033]
 [ 1.        ]]
diff_vector [-6.09136446e-05 -4.31181264e-05 -6.17676031e-05  0.00000000e+00]
b'Ready'
lower_point:  (1116, 512)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  511
non_zero_indexes:  [1117 1118]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1118
difference:  9.495548534578997e-05
TCP Position in Robot Coordinates:  [-0.44278148985

In [24]:
paint_image()
line=None
while (line!="ready"):
    line=input(Fore.GREEN+"Switch to local mode on robot, and set third  point.\n Then switch back to remote mode and input 'ready' to proceed:\n"+Style.RESET_ALL)


lower_point:  (1117, 511)
Switch to local mode on robot, and set third  point.
 Then switch back to remote mode and input 'ready' to proceed:
ready


In [25]:
### calibration_rotation_rpy_3
# Move to calibration position
ur.sendMoveL([start_position_tcp[0],start_position_tcp[1],start_position_tcp[2], \
              calibration_rotation_rpy_3[0], \
              calibration_rotation_rpy_3[1], \
              calibration_rotation_rpy_3[2]],
             calibration_TCP, 30 / 1000, 10,0)
ur.waitForMessage()

points_for_set_tcp.append(go_to_startposition(ur,calibration_position_for_needle_end,calibration_rotation_rpy_3,temporal_transformation_matrix,socket))
# Get actual robot data
ur.formatSharedData()



b'Ready'

 start_position_3d_t [[ 0.0042571]
 [-0.0136484]
 [ 0.2102571]
 [ 1.       ]]

 start_position_3d_rob_coord_t [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]]
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  516
non_zero_indexes:  [1123]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1123
difference:  0.0006159117475267761
TCP Position in Robot Coordinates:  [-0.4436128145359022, -0.28609028334793607, 0.01419950246490953]
next point:  -0.44378194739544397 -0.28588419353452754 0.014353638394587972
start_position_3d_rob_coord_t
 [[-0.44367709]
 [-0.28614515]
 [ 0.0144621 ]
 [ 1.        ]] 
actual_position_3d_rob_coord: 
 [[-0.44333883]
 [-0.28655733]
 [ 0.01415383]
 [ 1.        ]]
diff_vector [ 0.00033827 -0.00041218 -0.00030827  0.        ]
b'Ready'
lower_point:  (1123, 516)
Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  512
non_zero_indexes:  [1119 1121]
non_zer

In [26]:
print("use this points for set tcp:\n")
print(points_for_set_tcp[0])
print(points_for_set_tcp[1])
print(points_for_set_tcp[2])
print(points_for_set_tcp[3])

use this points for set tcp:

[[-0.4440039666057461, -0.2856965557161765, 0.0145722860576607, 0.023578347481803645, 2.963778472339572, -0.27290848284227986]]
[-0.44441119235204646, -0.2860571984597059, 0.01427988382250675, 0.0034297275779193725, 2.771571664545744, 0.00011248759118624279]
[-0.44279539434831583, -0.28603754569146367, 0.013971689751750838, -0.00321572328446737, -2.8046012076956153, 0.0016111304206092559]
[-0.4439848933204733, -0.2857082619305109, 0.014529816869140122, 0.023547598137331132, 2.963735051791385, -0.2729473239250163]


In [27]:
ur.ActualTcpPos

[-0.4440039666057461,
 -0.2856965557161765,
 0.0145722860576607,
 0.023578347481803645,
 2.963778472339572,
 -0.27290848284227986]

In [28]:
paint_image()
line=None
while (line!="ready"):
    line=input(Fore.GREEN+"Switch to local mode on robot, and set  fourth point.\n Calculate and set new TCP.\nThen switch back to remote mode and input 'ready' to proceed:\n"+Style.RESET_ALL)


lower_point:  (1117, 511)
Switch to local mode on robot, and set  fourth point.
 Calculate and set new TCP.
Then switch back to remote mode and input 'ready' to proceed:
ready


# Final calibration 
## Stop program here and type in detected tcp for final calibration

In [34]:
calibration_TCP=[0.44/1000,1.62/1000,232.32/1000] #[4.38/1000,0.77/1000,230.91/1000]

In [35]:
picked_points=[]
picked_points_3d_camera=[]
picked_points_3d_robot=[]
picked_points_3d_robot_current=[]


In [36]:
### known point in robot coordinates
# reference_point will be used to track improvement or disimprovement of calibration. It known from previous 
# iteration and is used for expretiment only 
reference_point=[-0.214588036495642, -0.4302389923186814, 0.08]
rejected=0
distance=0.005
try:
    for waypoint in tqdm(calibration_waypoints_final):

        # Move to next point and get coordinates
        ur.sendMoveL([waypoint[0],waypoint[1],waypoint[2], \
                      start_position_rotation_rpy[0], \
                      start_position_rotation_rpy[1], \
                      start_position_rotation_rpy[2]],
                     calibration_TCP, 30 / 1000, 0.2,0)
        ur.waitForMessage()

        # Get actual robot data
        ur.formatSharedData()

        # Give robot time to stop vibration
        time.sleep(0.5)

        # Get 3d and color frames
        wire_1_mat,grayimage=get_images(socket)
        reference_point_in_camera_coordinates,lower_point= find_lower_pixel(wire_1_mat)
        print("reference_point_in_camera_coordinates: ",reference_point_in_camera_coordinates,"\n")
        try:
            if len(picked_points_3d_robot)<3:
                ur.formatSharedData()
                picked_points_3d_camera.append(reference_point_in_camera_coordinates)
                picked_points_3d_robot.append([ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]])
                
            else:
                temp_points_camera=picked_points_3d_camera.copy()
                temp_points_robot=picked_points_3d_robot.copy()
                temp_points_camera.append(reference_point_in_camera_coordinates)
                temp_points_robot.append([ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]])
                transformation_matrix_temp=create_transformation_matrix(temp_points_camera, temp_points_robot)
                temp_point=np.append(reference_point_in_camera_coordinates,1).reshape((-1))
                temp_point=np.dot(transformation_matrix_temp,temp_point)[:3]
                path=reference_point[:2]-temp_point[:2]
                temp_point=[ur.ActualTcpPos[0]+path[0],ur.ActualTcpPos[1]+path[1]]
                print()
                distance_current=euclidean_distances([temp_point[:2]], [reference_point[:2]])

                
                
                if  distance_current<0.00011 or distance_current < distance :
                    print("distance_current: ",distance_current)
                    distance=distance-(distance-distance_current)/3
                    print("new distance: ",distance)
                    # Append next detected point to arrays with both coordinate systems
                    picked_points_3d_camera.append(reference_point_in_camera_coordinates)
                    picked_points_3d_robot.append([ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]])
                    paint_image()
                else: 
                    rejected+=1
                    print("\n\n\n bigger: distance",distance_current," len(picked_points_3d_robot): ",
                          len(picked_points_3d_robot))
                    
        except Exception:
            print("error, continue")
            print(traceback.format_exc())
            continue

        #paint_image()
        frame_num+=1

    if picked_points_3d_camera==[] or picked_points_3d_robot==[]:
        print("Arrays with points are empty. Probably aruco symbol is out of FOV. Calibration is impossible")
        exit(1)

    transformation_matrix=create_transformation_matrix(picked_points_3d_camera, picked_points_3d_robot)

    with open("transformation_matrix_Photoneo.pkl", 'wb') as handle:
        pkl.dump(transformation_matrix, handle, protocol=4)
        print("Transformation matrix saved in: ","transformation_matrix_Photoneo.pkl")

finally:
    print("rejected points: ",rejected, " from ",len(calibration_waypoints_final))
    cv2.destroyAllWindows()

  0%|          | 0/102 [00:00<?, ?it/s]

b'Ready'


  1%|          | 1/102 [00:17<30:07, 17.90s/it]

Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  444
non_zero_indexes:  [565 566]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  566
reference_point_in_camera_coordinates:  [-0.0263148 -0.0162916  0.193925 ] 

b'Ready'


  2%|▏         | 2/102 [00:34<28:45, 17.26s/it]

...

Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  406
non_zero_indexes:  [1269]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1269
reference_point_in_camera_coordinates:  [ 0.0133585  -0.020003    0.21145649] 

Transformation matrix created.
[[ 0.99955538  0.00575851  0.02925553 -0.45442724]
 [-0.02907435 -0.02936741  0.99914575 -0.49492292]
 [ 0.00661275 -0.9995521  -0.02918693  0.00454662]
 [ 0.          0.          0.          1.        ]]
rotations rX,rY,rZ:  -91.67256174842726 -0.37888520512985646 -1.6661086935250893




 bigger: distance [[0.00020832]]  len(picked_points_3d_robot):  38
b'Ready'


 99%|█████████▉| 101/102 [28:18<00:17, 17.06s/it]

Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  300
non_zero_indexes:  [754]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  754
reference_point_in_camera_coordinates:  [-0.0163285 -0.0247005  0.1980696] 

Transformation matrix created.
[[ 0.99955608  0.00589494  0.02920428 -0.45441142]
 [-0.02902155 -0.02901063  0.99915771 -0.49492475]
 [ 0.00673721 -0.99956172 -0.02882667  0.00447303]
 [ 0.          0.          0.          1.        ]]
rotations rX,rY,rZ:  -91.65191305609196 -0.38601652786962365 -1.6630835234593098




 bigger: distance [[0.00012283]]  len(picked_points_3d_robot):  38
b'Ready'


100%|██████████| 102/102 [28:35<00:00, 16.82s/it]

Received reply from server: b'1'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
last_non_zero_height:  279
non_zero_indexes:  [1645 1646]
non_zero_indexes[int(len(non_zero_indexes)/2)]:  1646
reference_point_in_camera_coordinates:  [ 0.0339035 -0.0260791  0.199534 ] 

Transformation matrix created.
[[ 0.99954952  0.00584089  0.0294386  -0.45446672]
 [-0.02925654 -0.02912867  0.99914742 -0.49492508]
 [ 0.00669341 -0.9995586  -0.02894466  0.00450039]
 [ 0.          0.          0.          1.        ]]
rotations rX,rY,rZ:  -91.65867591200116 -0.3835072698794423 -1.6765532166350052




 bigger: distance [[0.0001533]]  len(picked_points_3d_robot):  38
Transformation matrix created.
[[ 0.99955423  0.00579947  0.02928669 -0.45443356]
 [-0.02910513 -0.029243    0.99914851 -0.49492601]
 [ 0.00665097 -0.99955551 -0.02906117  0.00452467]
 [ 0.          0.          0.          1.        ]]
rotations rX,rY,rZ:  -91.66535362589488 -0.3810750968188738 -1.66787379606247
Transformation matrix saved in:  tr

In [37]:
ur.stopUpdateProzess()

### Plot and save taken points 

In [38]:
camera_points = o3d.geometry.PointCloud()
camera_points.points=o3d.utility.Vector3dVector(picked_points_3d_camera)
camera_points.paint_uniform_color([0.9, 0.0, 0.0])

robot_points = o3d.geometry.PointCloud()
robot_points.points=o3d.utility.Vector3dVector(picked_points_3d_robot)
robot_points.paint_uniform_color([0.0, 0.9, 0.0])

camera_points_t=[]
for temp_point in picked_points_3d_camera:

    temp_point_t_1=np.append(temp_point,1).reshape((-1))

    camera_points_t.append(np.dot(transformation_matrix,temp_point_t_1)[:3].tolist())
camera_points_transformed = o3d.geometry.PointCloud()
camera_points_transformed.points=o3d.utility.Vector3dVector(camera_points_t)
camera_points_transformed.paint_uniform_color([0.0, 0.0, 0.9])

o3d.visualization.draw_geometries([camera_points,robot_points,camera_points_transformed])

In [39]:
print("MAE: ",mean_absolute_error(picked_points_3d_robot, camera_points_t))

MAE:  9.370838575669885e-05


In [40]:
combined_cloud = o3d.geometry.PointCloud()
combined_cloud=camera_points+robot_points+camera_points_transformed
o3d.io.write_point_cloud("images/calibration_points.ply", combined_cloud)

True